# PatchTST for the forecast of German Energy consumption

TODO
- Import necessary code
- See Nick's structure and write a similar notebook with similar sections.
- Check in original PatchTST repo code where are the preds
- check to see if I can just discard the first 56
- run my model with my dataset and with regular dataset but discarding 56 outputs and see which is more accurate
- If I can't find it, just run the model with my code, verify it works (how?) and stick to that way of windowing
- Create new dataset class for the supermarket data (gap?)
- Order everything in this notebook

In [ ]:
import random

import numpy as np
import pandas as pd
import torch
import os

import datetime
from deutschland import feiertage
from deutschland.feiertage.api import default_api
configuration = feiertage.Configuration(
    host = "https://feiertage-api.de/api"
)

import PatchTST
from utils.tools import EarlyStopping, visual


## Loading data

In [ ]:
# deal with data loading later. For the time being, we use data already stored as .csv
data_path = './dataset/'  # path to the data file
root_path_name=data_path
data_path_name='SMARD_converted.csv'

In [ ]:
path = os.path.abspath(data_path)
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
df_SMARD = pd.DataFrame()

url="https://raw.githubusercontent.com/koljaeger/smardcast/main/data/Realisierter_Stromverbrauch_"

for year in range(2015, 2023+1, 1):
    df_SMARD = pd.concat([df_SMARD, pd.read_csv(url+str(year)+"01010000_"+str(year)+"12312359_Viertelstunde.csv", 
                                    delimiter=";", thousands='.', decimal=",", dtype={"Datum":str})], axis=0, ignore_index=True)
    
df_SMARD["date"] = pd.to_datetime(df_SMARD.pop("Datum")+' '+df_SMARD.pop("Anfang"), format="%d.%m.%Y %H:%M")
df_SMARD["date"] = df_SMARD["date"].dt.tz_localize("Europe/Berlin", ambiguous='infer').dt.tz_convert('UTC')

df_SMARD = df_SMARD.rename(
        columns={
        # 'Datum': "date", # expected by Dataset_Custom
        'Gesamt (Netzlast) [MWh] Originalauflösungen': 'OT', # Grid load is target feature
        'Residuallast [MWh] Originalauflösungen': 'Residual_load',
        'Pumpspeicher [MWh] Originalauflösungen' : 'Pumped_storage'
        }
    )

# "Ende" and "Pumped_storage" are not needed
df_SMARD = df_SMARD.drop(columns=["Ende", "Pumped_storage", "Residual_load"])

# Add integer to indicate day of week 
df_SMARD["Weekday"] = [n.day_of_week for n in df_SMARD["date"]]

# Add timestep functions
minute = 60
hour = 60*minute
day = 24*hour
week = 7*day
year = (365.2425)*day
month = year/12

date_time = pd.to_datetime(df_SMARD["date"], format='%Y-%m-%d %H:%M:%S')
timestamp_s = date_time.map(pd.Timestamp.timestamp)

df_SMARD['Hour sin'] = np.sin(timestamp_s * (2 * np.pi / hour))
df_SMARD['Hour cos'] = np.cos(timestamp_s * (2 * np.pi / hour))
df_SMARD['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df_SMARD['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df_SMARD['Week sin'] = np.sin(timestamp_s * (2 * np.pi / week))
df_SMARD['Week cos'] = np.cos(timestamp_s * (2 * np.pi / week))
df_SMARD['Month sin'] = np.sin(timestamp_s * (2 * np.pi / month))
df_SMARD['Month cos'] = np.cos(timestamp_s * (2 * np.pi / month))
df_SMARD['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df_SMARD['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

# Add holiday column
# Assuming df["date"] contains pandas datetime objects
lowest_year = df_SMARD["date"].dt.year.min()
highest_year = df_SMARD["date"].dt.year.max()

holiday_dates = []

with feiertage.ApiClient(configuration) as api_client:
    api_instance = default_api.DefaultApi(api_client)

for jahr in range(lowest_year, highest_year+1):
    try:
        api_response = api_instance.get_feiertage(jahr=str(jahr), nur_land="HH", nur_daten=1)
    except feiertage.ApiException as e:
        print("Exception when calling DefaultApi->get_feiertage: %s\n" % e)

    for feiertag in api_response:
        holiday_dates.append(api_response[feiertag])

df_SMARD["date"] = pd.to_datetime(df_SMARD["date"], format='%Y-%m-%d %H:%M:%S')
df_days = df_SMARD["date"].dt.date

# Calculate the days before holidays
days_before_holidays = {date - datetime.timedelta(days=1) for date in holiday_dates}

# Mark holidays and days before holidays in the DataFrame
df_SMARD['Is Holiday'] = df_days.apply(lambda x: 1 if x in holiday_dates else (0.5 if x in days_before_holidays else 0))
print("Holiday Signal Added!")

# Move 'OT' to the last position
cols = [c for c in df_SMARD.columns if c != 'OT'] + ['OT']
df_SMARD = df_SMARD[cols]

df_SMARD.to_csv(data_path+data_path_name, index=False)

PyTorch uses dataset and dataload classes to handle data for the model. In order to prepare for these classes, we load the data and store it as a .csv file

In [ ]:
# Set seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Parameters

#PatchTST parameters
seq_len=672 #Context window
pred_len=96 #Forecast horizon
patch_len=16
stride=8
#PatchTST defaults
enc_in=1
e_layers=3
n_heads=16
d_model=128
d_ff=256
dropout=0.2
fc_dropout=0.2
head_dropout=0
individual_head=0 #True 1 False 0
padding_patch='end' #end: padding on the end
revin=1 # True 1 False 0
affine=0 # True 1 False 0
subtract_last=0 # 0: subtract mean; 1: subtract last
decomposition=0 # decomposition; True 1 False 0
kernel_size=25 
scale = True

dataset = 'SMARD_raw'

# Training parameters
batch_size=32 #default 128
learning_rate=0.0001
num_epochs=100
label_len=0 #start token length | default 48 
pct_start=0.3 # for cosine warmup
patience = 20
features='S' # 'S' single variable; 'MS' multivariate
debugging=False
if debugging:
    num_epochs=2
    batch_size=2
    PatchTST.e_layers=1
    PatchTST.d_model=8
    PatchTST.d_ff=16
    PatchTST.n_heads=2
    num_workers=0
else:
    num_workers=10
# drop_last=True


In [ ]:
from argparse import Namespace

configs = Namespace(
    enc_in=enc_in,
    seq_len=seq_len,
    pred_len=pred_len,
    e_layers=e_layers,
    n_heads=n_heads,
    d_model=d_model,
    d_ff=d_ff,
    dropout=dropout,
    fc_dropout=fc_dropout,
    head_dropout=head_dropout,
    individual=individual_head,
    patch_len=patch_len,
    stride=stride,
    padding_patch=padding_patch,
    revin=revin,
    affine=affine,
    subtract_last=subtract_last,
    decomposition=decomposition,
    kernel_size=kernel_size  
)


In [ ]:
from datetime import date, datetime
setting = f'{data_path_name}_ft{features}_sl{seq_len}_ll{label_len}_pl{pred_len}_dm{d_model}_nh{n_heads}_el{e_layers}_df{d_ff}_ds{dropout}_eb{batch_size}_lr{dataset}+sc{scale}+{datetime.today()}'

In [ ]:
from datasets import Dataset_Custom, Dataset_SMARD

# Load data
train_data = Dataset_SMARD(root_path=root_path_name,
                            data_path=data_path_name,
                            flag='train',
                            size=[seq_len, label_len, pred_len],
                            features=features, 
                            target='OT',
                            split_mode='fixed',
                            scale=scale)

val_data = Dataset_SMARD(root_path=root_path_name,
                            data_path=data_path_name,
                            flag='val',
                            size=[seq_len, label_len, pred_len],
                            features=features, 
                            target='OT',
                            split_mode='fixed',
                            scale=scale)

test_data = Dataset_SMARD(root_path=root_path_name,
                            data_path=data_path_name,
                            flag='test',
                            size=[seq_len, label_len, pred_len],
                            features=features, 
                            target='OT',
                            split_mode='fixed',
                            scale=scale)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True, ## Should this be false?
        num_workers=num_workers,
        drop_last=True)

val_loader = DataLoader(
        val_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True)

test_loader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True)

In [ ]:
for i in range(3):
    seq_x, seq_y = test_data[i]
    print(f"Sample {i} — seq_x[0:3]:\n", seq_x[:3])
    print(f"Sample {i} — seq_y[0:3]:\n", seq_y[:3])


In [ ]:
examples = iter(test_loader)
example_batch = next(examples)
print(f"shape of this example: {example_batch[0].shape}")
print(f"num of examples in each batch: {len(example_batch)}")

print(f"num of batches: {len(examples)}")


In [ ]:

# Initialize model
model = PatchTST.Model(configs).to(device)
print(model)
path = os.path.join('./checkpoints/', setting)
if not os.path.exists(path):
    os.makedirs(path)

train_steps = len(train_loader)
# Define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer = optimizer,
                                            steps_per_epoch = train_steps,
                                            pct_start = pct_start,
                                            epochs = num_epochs,
                                            max_lr = learning_rate)
early_stopping = EarlyStopping(patience=patience, verbose=True)


In [ ]:
def vali(vali_loader, criterion):
        total_loss = []
        model.eval()
        with torch.no_grad():
            for i, (batch_x, batch_y) in enumerate(vali_loader):
                batch_x = batch_x.float().to(device)
                batch_y = batch_y.float().to(device)

                outputs = model(batch_x)
                f_dim = -1 if features == 'MS' else 0
                outputs = outputs[:, -pred_len:, f_dim:]
                batch_y = batch_y[:, -pred_len:, f_dim:].to(device)

                pred = outputs.detach().cpu()
                true = batch_y.detach().cpu()

                loss = criterion(pred, true)

                total_loss.append(loss)
        total_loss = np.average(total_loss)
        model.train()
        return total_loss

In [ ]:
# Train the model

import time

for epoch in range(num_epochs):
    iter_count = 0
    train_loss = []
    time_now = time.time()
    model.train()
    epoch_time = time.time()
    for i, (batch_x, batch_y) in enumerate(train_loader):
        iter_count += 1
        optimizer.zero_grad()
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
    
        outputs = model(batch_x)
        # print(outputs.shape,batch_y.shape)
        f_dim = -1 if features == 'MS' else 0
        outputs = outputs[:, -pred_len:, f_dim:]
        batch_y = batch_y[:, -pred_len:, f_dim:].to(device)
        loss = criterion(outputs, batch_y)
        train_loss.append(loss.item())

        if (i + 1) % 100 == 0:
            print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
            speed = (time.time() - time_now) / iter_count
            left_time = speed * ((num_epochs - epoch) * train_steps - i)
            print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
            iter_count = 0
            time_now = time.time()

        loss.backward()
        optimizer.step()
            
        #Adjust learning rate
        lr_adjust = {epoch: scheduler.get_last_lr()[0]}
        if epoch in lr_adjust.keys():
            lr = lr_adjust[epoch]
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
            if False: print('Updating learning rate to {}'.format(lr))
        scheduler.step()

    print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
    train_loss = np.average(train_loss)
    vali_loss = vali(val_loader, criterion)
    test_loss = vali(test_loader, criterion)

    print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f} Vali Loss: {3:.7f} Test Loss: {4:.7f}".format(
        epoch + 1, train_steps, train_loss, vali_loss, test_loss))
    early_stopping(vali_loss, model, path)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    print('Updating learning rate to {}'.format(scheduler.get_last_lr()[0]))

In [ ]:
#Testing the model
from utils.metrics import metric

best_model_path = path + '/' + 'checkpoint.pth'
model.load_state_dict(torch.load(best_model_path))

def test(loader):
    preds = []
    trues = []
    inputx = []
    mode = 'test' if loader == test_loader else 'val'

    folder_path = './test_results/' + setting + '/' + mode + '/'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    model.eval()
    with torch.no_grad():
        for i, (batch_x, batch_y) in enumerate(loader):
            batch_x = batch_x.float().to(device)
            batch_y = batch_y.float().to(device)

            outputs = model(batch_x)
            f_dim = -1 if features == 'MS' else 0
            # print(outputs.shape,batch_y.shape)
            # print(f"outputs \n {outputs}, batch_y (labels?) \n {batch_y}")
            outputs = outputs[:, -pred_len:, f_dim:]
            batch_y = batch_y[:, -pred_len:, f_dim:].to(device)
            outputs = outputs.detach().cpu().numpy()
            batch_y = batch_y.detach().cpu().numpy()

            pred = outputs  # outputs.detach().cpu().numpy()  # .squeeze()
            true = batch_y  # batch_y.detach().cpu().numpy()  # .squeeze()

            

            preds.append(pred)
            trues.append(true)
            inputx.append(batch_x.detach().cpu().numpy())
            if i % 20 == 0:
                input = batch_x.detach().cpu().numpy()
                gt = np.concatenate((input[0, :, -1], true[0, :, -1]), axis=0)
                pd = np.concatenate((input[0, :, -1], pred[0, :, -1]), axis=0)
                visual(gt, pd, os.path.join(folder_path, str(i) + '.pdf'))

    preds = np.array(preds)
    trues = np.array(trues)
    inputx = np.array(inputx)

    preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])
    trues = trues.reshape(-1, trues.shape[-2], trues.shape[-1])
    inputx = inputx.reshape(-1, inputx.shape[-2], inputx.shape[-1])

    # result save
    folder_path = './results/' + setting + '/' + mode + '/'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # mae, mse, rmse, mape, mspe, rse, corr = metric(preds, trues)
    # print('mse:{}, mae:{}, mape:{}'.format(mse, mae, mape))
    # f = open("result.txt", 'a')
    # f.write(setting + "  \n")
    # f.write('mse:{}, mae:{}, mape:{}'.format(mse, mae, mape))
    # f.write('\n')
    # f.write('\n')
    # f.close()

    MAPE_val = (abs((trues - preds)/trues)*100)
    print(f"MAPE_val: {MAPE_val.mean()}")

    # np.save(folder_path + 'metrics.npy', np.array([mae, mse, rmse, mape, mspe,rse, corr]))
    np.save(folder_path + mode + 'pred.npy', preds)
    np.save(folder_path + mode +'true.npy', trues)

    return preds, trues, inputx


In [ ]:
val_preds, val_trues, val_inputx = test(val_loader)
test_preds, test_trues, test_inputx = test(test_loader)

In [ ]:
print(val_preds.shape, val_trues.shape, val_inputx.shape)

In [ ]:
continuous_preds = val_preds.reshape(-1, val_preds.shape[-1])
continuous_trues = val_trues.reshape(-1, val_trues.shape[-1])


In [ ]:
MAPE = (abs((continuous_trues - continuous_preds)/continuous_trues)*100)
print(f"Validation MAPE: {MAPE.mean()}")

In [ ]:
continuous_preds = test_preds.reshape(-1, test_preds.shape[-1])
continuous_trues = test_preds.reshape(-1, test_preds.shape[-1])

In [ ]:
MAPE = (abs((continuous_trues - continuous_preds)/continuous_trues)*100)
print(f"Validation MAPE: {MAPE.mean()}")

In [ ]:
print(MAPE)
print(continuous_trues)
print(continuous_preds)


In [ ]:
# val_preds_inv = val_data.inverse_transform(val_preds)
# val_trues_inv = val_data.inverse_transform(val_trues)
# test_preds_inv = test_data.inverse_transform(test_preds)
# test_trues_inv = test_data.inverse_transform(test_trues)

- Make sure MAPE calculation is correct
- Ensure preds are preds 
- Do inverse transform
